In [90]:
import numpy as np
import pandas as pd
from pandas import DataFrame as df
from pandas import ExcelWriter
from pandas import ExcelFile
import itertools
import scipy.optimize as opt
from scipy.optimize import minimize
from scipy.optimize import differential_evolution
import geopy
from geopy import distance

In [128]:
# 1.reading 
df = pd.read_csv("radio_merger_data.csv")
#2.Convert scale of price and population to 1000 dollars and 1000 people respectively
df.price = df.price/1000
df.population_target = df.population_target/1000

In [129]:
# Defining counterfactual dataset for each year:
df2007 = df[df['year']==2007]
df2008 = df[df['year']==2008]
cf2007 = counterfact[counterfact['year']==2007]
cf2008 = counterfact[counterfact['year']==2008]

In [139]:
dfy = dict(iter(df.groupby('year', as_index = False)))
dfy[2007].describe().to_csv("describe2007.csv")
dfy[2008].describe().to_csv("describe2008.csv")
B = ['year', 'buyer_id', 'buyer_lat', 'buyer_long', 'num_stations_buyer', 'corp_owner_buyer']
T = ['target_id', 'target_lat', 'target_long', 'price', 'hhi_target', 'population_target']
data = [pd.DataFrame(dfy[2007]), pd.DataFrame(dfy[2008])]
counterfact = pd.DataFrame()
counterfact = pd.DataFrame([x[B].iloc[i].values.tolist() + x[T].iloc[j].values.tolist() for x in data for i in range(len(x)) for j in range(len(x)) if i!= j], columns = B + T)

In [140]:
counterfact.head()

,year,buyer_id,buyer_lat,buyer_long,num_stations_buyer,corp_owner_buyer,target_id,target_lat,target_long,price,hhi_target,population_target
0,2007.0,1.0,46.592512,-92.549564,3.0,0.0,2.0,33.025375,-86.059702,1472.46320,376.0,11.539
1,2007.0,1.0,46.592512,-92.549564,3.0,0.0,3.0,31.122499,-87.766408,3786.33390,129.0,182.265
2,2007.0,1.0,46.592512,-92.549564,3.0,0.0,4.0,36.196946,-94.006823,473.29174,188.0,203.065
3,2007.0,1.0,46.592512,-92.549564,3.0,0.0,5.0,40.909898,-73.457023,1840.57900,284.0,1493.350
4,2007.0,1.0,46.592512,-92.549564,3.0,0.0,6.0,39.269849,-107.023350,1104.34740,51.0,17.148


In [142]:
# define distance
def dist(d):
    '''
    This function calculates distance between two points in miles
    b_loc = The coordinates for buyer's location
    t_loc = The coordinates for target's location
    '''
    b_loc = (d['buyer_lat'], d['buyer_long'])
    t_loc = (d['target_lat'], d['target_long'])
    return distance.distance(b_loc, t_loc).miles

df['distance'] = df.apply(dist, axis=1)
counterfact['distance'] = counterfact.apply(dist, axis=1)


In [143]:
#Score function definition for model 1 , without transfers, using Gale-Shapely Algorithm
def score1_GS(params, m, n):
    '''
    Payoff function for mergers to be used in the indicator function
    Indicator == 1 if f(b,t) + f(b',t') > f(b',t) + f(b,t')
    Indicator == 0 otherwise
    '''
    f_b_t = m['num_stations_buyer'] * m['population_target'] + params[0] * m['corp_owner_buyer'] * m['population_target'] + params[1] * m['distance']
    f_cb_ct = n['num_stations_buyer'] * n['population_target'] + params[0] * n['corp_owner_buyer'] * n['population_target'] + params[1] * n['distance']
    f_cb_t = n['num_stations_buyer'] * m['population_target'] + params[0] * n['corp_owner_buyer'] * m['population_target'] + params[1] * m['distance']
    f_b_ct = m['num_stations_buyer'] * n['population_target'] + params[0] * m['corp_owner_buyer'] * n['population_target'] + params[1] * m['distance']
    L = f_b_t + f_cb_ct
    R = f_cb_t + f_b_ct
    indicator=(L>R)
    total_payoff = indicator.sum()
    return -total_payoff

bounds = [(-10,10), (-10,10)]
GS1_dif = differential_evolution(score1_GS, bounds, args=(df, counterfact), strategy='best1bin', maxiter=10000)

In [144]:
#Score function definition for model 2 , without transfers, using Gale-Shapely Algorithm
def score2_GS(params, m, n):

    '''
    Payoff function for mergers to be used in the indicator function
    Indicator == 1 if f(b,t) + f(b',t') > f(b',t) + f(b,t')
    Indicator == 0 otherwise
    '''
    f_b_t = params[0] * m['num_stations_buyer'] * m['population_target'] + params[1] * m['corp_owner_buyer'] * m['population_target'] + params[2] * m['hhi_target'] + params[3] * m['distance']
    f_cb_ct = params[0] * n['num_stations_buyer'] * n['population_target'] + params[1] * n['corp_owner_buyer'] * n['population_target'] + params[2]*n['hhi_target'] + params[3] * n['distance']
    f_cb_t = params[0] * n['num_stations_buyer'] * m['population_target'] + params[1] * n['corp_owner_buyer'] * m['population_target'] + params[2] * m['hhi_target'] + params[3] * m['distance']
    f_b_ct = params[0] * m['num_stations_buyer'] * n['population_target'] + params[1] * n['corp_owner_buyer'] * n['population_target'] + params[2] * m['hhi_target'] + params[3] * m['distance']

    L = f_b_t + f_cb_ct
    R = f_cb_t + f_b_ct
    indicator=(L>R)
    total_payoff = indicator.sum()
    return -total_payoff

bounds = [(-5,5), (-10,10), (-10, 10), (-10,10)]
GS2_dif = differential_evolution(score2_GS, bounds, args=(df, counterfact), strategy='best1bin', maxiter=100000)

In [145]:
#Score function definition for model 1 , with transfers, using Becker-Shapely-Subik Algorithm
def score1_BSS(params, m, n):
    '''
    Payoff function for mergers to be used in the indicator function
    Indicator == 1 if f(b,t) + f(b',t') > f(b',t) + f(b,t')
    Indicator == 0 otherwise
    '''
    f_b_t = m['num_stations_buyer'] * m['population_target'] + params[0] * m['corp_owner_buyer'] * m['population_target'] + params[1] * m['distance']
    f_cb_ct = n['num_stations_buyer'] * n['population_target'] + params[0] * n['corp_owner_buyer'] * n['population_target'] + params[1] * n['distance']
    f_cb_t = n['num_stations_buyer'] * m['population_target'] + params[0] * n['corp_owner_buyer'] * m['population_target'] + params[1] * m['distance']
    f_b_ct = m['num_stations_buyer'] * n['population_target'] + params[0] * m['corp_owner_buyer'] * n['population_target'] + params[1] * m['distance']
    L1 = f_b_t - f_b_ct
    R1 = m['price'] - n['price']
    L2 = f_cb_ct - f_cb_t
    R2 = n['price'] - m['price']
    indicator= ((L1 >= R1) & (L2 >= R2))
    total_payoff = indicator.sum()
    return -total_payoff

bounds = [(-5,5), (-10,10)]
BSS1_dif = differential_evolution(score1_BSS, bounds, args=(df, counterfact), strategy='best1bin', maxiter=10000)



In [148]:
#Score function definition for model 2 , with transfers, using Becker-Shapely-Subik Algorithm
def score2_BSS(params, m, n):
    '''
    Payoff function for mergers to be used in the indicator function.
    Indicator == 1 if f(b,t) + f(b',t') > f(b',t) + f(b,t')
    Indicator == 0 otherwise.
    '''
    f_b_t = params[0] * m['num_stations_buyer'] * m['population_target'] + params[1] * m['corp_owner_buyer'] * m['population_target'] + params[2] * m['hhi_target'] + params[3] * m['distance']
    f_cb_ct = params[0] * n['num_stations_buyer'] * n['population_target'] + params[1] * n['corp_owner_buyer'] * n['population_target'] + params[2]*n['hhi_target'] + params[3] * n['distance']
    f_cb_t = params[0] * n['num_stations_buyer'] * m['population_target'] + params[1] * n['corp_owner_buyer'] * m['population_target'] + params[2] * m['hhi_target'] + params[3] * m['distance']
    f_b_ct = params[0] * m['num_stations_buyer'] * n['population_target'] + params[1] * n['corp_owner_buyer'] * n['population_target'] + params[2] * m['hhi_target'] + params[3] * m['distance']

    L1 = f_b_t - f_b_ct
    R1 = m['price'] - n['price']
    L2 = f_cb_ct - f_cb_t
    R2 = n['price'] - m['price']
    indicator= ((L1 >= R1) & (L2 >= R2))
    total_payoff = indicator.sum()
    return -total_payoff

bounds = [(-5,5), (-10,10), (-10, 10), (-10,10)]
BSS2_dif = differential_evolution(score2_BSS, bounds, args=(df, counterfact), strategy='best1bin', maxiter=10000)

In [214]:
from tabulate import tabulate
d = [GS1_dif.x ,BSS1_dif.x] 
print(tabulate(d,headers=["Alpha ","Beta"],tablefmt="github"))

|   Alpha  |    Beta |
|----------|---------|
|  7.26329 | 6.9054  |
| -2.1352  | 9.47624 |


In [222]:
dd=  [GS2_dif.x,BSS2_dif.x] 

print(tabulate(dd,headers=["Sigma "," Alpha ","Gamma ","Beta" ],tablefmt="github"))

|   Sigma  |    Alpha  |   Gamma  |    Beta |
|----------|-----------|----------|---------|
| 0.729169 |   7.3894  |  1.08678 | 4.95609 |
| 2.41002  |  -4.62312 | -3.21928 | 9.72082 |


In [218]:
from tabulate import tabulate
d = [GS1_dif.x ,BSS1_dif.x] 
print(tabulate(d,headers=["Alpha ","Beta"],tablefmt="latex"))

\begin{tabular}{rr}
\hline
   Alpha  &    Beta \\
\hline
  7.26329 & 6.9054  \\
 -2.1352  & 9.47624 \\
\hline
\end{tabular}


In [223]:
dd=  [GS2_dif.x,BSS2_dif.x] 

print(tabulate(dd,headers=["Sigma "," Alpha ","Gamma ","Beta" ],tablefmt="latex"))

\begin{tabular}{rrrr}
\hline
   Sigma  &    Alpha  &   Gamma  &    Beta \\
\hline
 0.729169 &   7.3894  &  1.08678 & 4.95609 \\
 2.41002  &  -4.62312 & -3.21928 & 9.72082 \\
\hline
\end{tabular}
